# SOFM保序特定验证

## 三角形区域

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import random
from math import sqrt
from numpy import linspace, zeros, amin, where
from PIL import Image
import os

class AnimatedGIFCreator:
    def __init__(self, gif_dir='temp_gif_frames', gif_file='output.gif'):
        self.gif_dir = gif_dir
        self.gif_file = gif_file
        self.frames = []
        self.frame_count = 0

        if not os.path.exists(gif_dir):
            os.makedirs(gif_dir)

    def add_plot_frame(self, plt_figure=None):
        if plt_figure is None:
            plt_figure = plt.gcf()

        filename = os.path.join(self.gif_dir, f"frame_{self.frame_count:04d}.png")
        plt_figure.savefig(filename, bbox_inches='tight')
        self.frames.append(Image.open(filename))
        self.frame_count += 1
        plt.close(plt_figure)

    def create_gif(self, duration=100, last_frame_duration=200):
        if not self.frames:
            print("No frames to save.")
            return

        durations = [duration] * (len(self.frames) - 1) + [last_frame_duration]
        self.frames[0].save(
            self.gif_file,
            save_all=True,
            append_images=self.frames[1:],
            duration=durations,
            loop=0
        )
        print(f"GIF saved as {self.gif_file}")

    def clear_frames(self):
        for frame in os.listdir(self.gif_dir):
            os.remove(os.path.join(self.gif_dir, frame))
        self.frames = []
        self.frame_count = 0

def generate_triangle_data(num_points):
    points = []
    for _ in range(num_points):
        while True:
            x = random.uniform(0, 1)
            y = random.uniform(0, sqrt(3) / 2)
            y_limit = sqrt(3) * (0.5 - abs(x - 0.5))
            if y > y_limit:
                continue
            points.append([x, y])
            break
    return np.array(points)

def plot_data(data, line_flag=0, plot_title=''):
    if line_flag == 0:
        plt.scatter(data[:, 0], data[:, 1], s=10, c='blue')
    else:
        plt.scatter(data[:, 0], data[:, 1], s=35, c='red')
        plt.plot(data[:, 0], data[:, 1], 'y-', linewidth=1)

    boundary_x = linspace(0, 1, 200)
    boundary_y = [sqrt(3) * (0.5 - abs(x - 0.5)) for x in boundary_x]
    plt.plot(boundary_x, boundary_y, 'c--', linewidth=1)
    plt.plot(boundary_x, zeros(len(boundary_x)), 'c--', linewidth=1)

    plt.xlabel("x1")
    plt.ylabel("x2")
    plt.axis([-0.05, 1.05, -0.05, .9])

    if plot_title:
        plt.title(plot_title)

    plt.grid(True)
    plt.tight_layout()

def find_winner(input_vector, weight_matrix):
    distances = np.array([(input_vector - weight).dot(input_vector - weight) for weight in weight_matrix])
    return list(where(distances == amin(distances)))[0][0]

def find_neighborhood_1d(winner_index, num_neurons, neighborhood_radius):
    if neighborhood_radius <= 0:
        return [winner_index]
    neighborhood = []
    for offset in range(-neighborhood_radius, neighborhood_radius + 1):
        if 0 <= winner_index + offset < num_neurons:
            neighborhood.append(winner_index + offset)
    return neighborhood

def update_weights(input_data, weight_matrix, learning_rate, neighborhood_radius):
    for input_vector in input_data:
        winner_index = find_winner(input_vector, weight_matrix)
        neighborhood = find_neighborhood_1d(winner_index, weight_matrix.shape[0], neighborhood_radius)
        for neuron_index in neighborhood:
            weight_matrix[neuron_index] += learning_rate * (input_vector - weight_matrix[neuron_index])
    return weight_matrix

configurations = [
    {'num_samples': 100, 'num_neurons': 15, 'num_iterations': 200, 'gif_file': 'C:\\Users\\ma\\Desktop\\新建文件夹\\人工神经网络\\第三次作业\\三角区域100点.gif'},
    {'num_samples': 200, 'num_neurons': 50, 'num_iterations': 200, 'gif_file': 'C:\\Users\\ma\\Desktop\\新建文件夹\\人工神经网络\\第三次作业\\三角区域200点.gif'},
    {'num_samples': 300, 'num_neurons': 100, 'num_iterations': 200, 'gif_file': 'C:\\Users\\ma\\Desktop\\新建文件夹\\人工神经网络\\第三次作业\\三角区域300点.gif'},
]

for config in configurations:
    num_samples = config['num_samples']
    num_neurons = config['num_neurons']
    num_iterations = config['num_iterations']
    gif_file = config['gif_file']

    eta_start = 0.3
    eta_end = 0.01
    radius_start = 10
    radius_end = 0

    sample_data = generate_triangle_data(num_samples)
    weights = np.random.rand(num_neurons, sample_data.shape[1])
    weights[:, 1] *= sqrt(3) / 2

    gif_creator = AnimatedGIFCreator(gif_dir='temp_gif_frames', gif_file=gif_file)

    for iteration in range(num_iterations):
        eta = (eta_start - eta_end) * (num_iterations - iteration) / (num_iterations - 1) + eta_end
        radius = int((radius_start - radius_end) * (num_iterations - iteration) / (num_iterations - 1) + radius_end)

        weights = update_weights(sample_data, weights, eta, radius)

        plt.clf()
        plot_data(sample_data, line_flag=0)
        plot_data(weights, line_flag=1, plot_title=f"Step: {iteration + 1}/{num_iterations}, Radius: {radius}, Eta: {eta:.2f}")
        plt.draw()
        gif_creator.add_plot_frame(plt.gcf())

    gif_creator.create_gif(duration=100, last_frame_duration=500)
    gif_creator.clear_frames()


GIF saved as C:\Users\ma\Desktop\新建文件夹\人工神经网络\第三次作业\三角区域100点.gif
GIF saved as C:\Users\ma\Desktop\新建文件夹\人工神经网络\第三次作业\三角区域200点.gif
GIF saved as C:\Users\ma\Desktop\新建文件夹\人工神经网络\第三次作业\三角区域300点.gif


## 矩形区域

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import amin, array, where, vstack
from PIL import Image
import os

class AnimatedGIFCreator:
    def __init__(self, gif_dir='temp_gif_frames', gif_file='output.gif'):
        self.gif_dir = gif_dir
        self.gif_file = gif_file
        self.frames = []
        self.frame_count = 0

        if not os.path.exists(gif_dir):
            os.makedirs(gif_dir)

    def add_plot_frame(self, plt_figure=None):
        if plt_figure is None:
            plt_figure = plt.gcf()

        filename = os.path.join(self.gif_dir, f"frame_{self.frame_count:04d}.png")
        plt_figure.savefig(filename, bbox_inches='tight')
        self.frames.append(Image.open(filename))
        self.frame_count += 1
        plt.close(plt_figure)

    def create_gif(self, duration=100, last_frame_duration=200):
        if not self.frames:
            print("No frames to save.")
            return

        durations = [duration] * (len(self.frames) - 1) + [last_frame_duration]
        self.frames[0].save(
            self.gif_file,
            save_all=True,
            append_images=self.frames[1:],
            duration=durations,
            loop=0
        )
        print(f"GIF saved as {self.gif_file}")

    def clear_frames(self):
        for frame in os.listdir(self.gif_dir):
            os.remove(os.path.join(self.gif_dir, frame))
        self.frames = []
        self.frame_count = 0

def generate_rectangle_data(num_points):
    x_coords = np.random.rand(num_points)
    y_coords = np.random.rand(num_points)
    data_points = vstack((x_coords, y_coords)).T
    return data_points

def plot_data(data_points):
    plt.scatter(data_points[:, 0], data_points[:, 1], s=10, c='blue', label='Training Data')
    plt.vlines(0, 0, 1, color='green', linewidth=1, linestyles='--')
    plt.vlines(1, 0, 1, color='green', linewidth=1, linestyles='--')
    plt.hlines(0, 0, 1, color='green', linewidth=1, linestyles='--')
    plt.hlines(1, 0, 1, color='green', linewidth=1, linestyles='--')
    plt.axis([-0.1, 1.1, -0.1, 1.2])
    plt.xlabel("x1")
    plt.ylabel("x2")
    plt.grid(True)

def plot_weights(weight_matrix):
    reshaped_weights = weight_matrix.reshape(-1, 2)
    plt.scatter(reshaped_weights[:, 0], reshaped_weights[:, 1], s=40, c='red', label='Nodes')

    for row in weight_matrix:
        plt.plot(row[:, 0], row[:, 1], color='red', linewidth=1, linestyle='--')
    for col in weight_matrix.transpose((1, 0, 2)):
        plt.plot(col[:, 0], col[:, 1], color='red', linewidth=1, linestyle='--')

    plt.legend(loc="upper right")
    plt.tight_layout()

def find_winner(input_vector, weight_matrix):
    distances = array([(input_vector - weight).dot(input_vector - weight) for weight in weight_matrix.reshape(-1, 2)])
    index = list(where(distances == amin(distances)))[0][0]
    col_index = index % weight_matrix.shape[1]
    row_index = index // weight_matrix.shape[1]
    return (col_index, row_index)

def find_neighborhood_2d(winner_index, num_rows, num_cols, neighborhood_radius):
    if neighborhood_radius <= 0:
        return [winner_index]

    neighborhood = []
    winner_col, winner_row = winner_index

    for row_offset in range(-neighborhood_radius, neighborhood_radius + 1):
        if winner_row + row_offset < 0 or winner_row + row_offset >= num_rows:
            continue

        for col_offset in range(-neighborhood_radius, neighborhood_radius + 1):
            if winner_col + col_offset < 0 or winner_col + col_offset >= num_cols:
                continue
            neighborhood.append((winner_col + col_offset, winner_row + row_offset))

    return neighborhood

def update_weights(input_data, weight_matrix, learning_rate, neighborhood_radius):
    for input_vector in input_data:
        winner_index = find_winner(input_vector, weight_matrix)
        neighborhood = find_neighborhood_2d(winner_index, weight_matrix.shape[0], weight_matrix.shape[1], neighborhood_radius)

        for neighbor_index in neighborhood:
            col_index, row_index = neighbor_index
            weight_matrix[row_index, col_index] += learning_rate * (input_vector - weight_matrix[row_index, col_index])

    return weight_matrix

configurations = [
    {'num_samples': 500, 'grid_size': (10, 10), 'num_iterations': 100, 'eta_start': 0.1, 'eta_end': 0.01, 'radius_start': 3, 'radius_end': 0, 'gif_file': 'C:\\Users\\ma\\Desktop\\新建文件夹\\人工神经网络\\第三次作业\\矩形区域500点.gif'},
    {'num_samples': 200, 'grid_size': (10, 10), 'num_iterations': 100, 'eta_start': 0.1, 'eta_end': 0.01, 'radius_start': 3, 'radius_end': 0, 'gif_file': 'C:\\Users\\ma\\Desktop\\新建文件夹\\人工神经网络\\第三次作业\\矩形区域200点.gif'},
    {'num_samples': 2000, 'grid_size': (10, 10), 'num_iterations': 100, 'eta_start': 0.1, 'eta_end': 0.01, 'radius_start': 3, 'radius_end': 0, 'gif_file': 'C:\\Users\\ma\\Desktop\\新建文件夹\\人工神经网络\\第三次作业\\矩形区域2000点.gif'},
    {'num_samples': 200, 'grid_size': (5, 5), 'num_iterations': 100, 'eta_start': 0.1, 'eta_end': 0.01, 'radius_start': 3, 'radius_end': 0, 'gif_file': 'C:\\Users\\ma\\Desktop\\新建文件夹\\人工神经网络\\第三次作业\\矩形区域200点小区域.gif'},
]

for config in configurations:
    num_samples = config['num_samples']
    grid_rows, grid_cols = config['grid_size']
    num_neurons = grid_rows * grid_cols
    num_iterations = config['num_iterations']
    eta_start = config['eta_start']
    eta_end = config['eta_end']
    radius_start = config['radius_start']
    radius_end = config['radius_end']
    gif_file = config['gif_file']

    sample_data = generate_rectangle_data(num_samples)
    weights = np.stack((np.random.rand(grid_rows, grid_cols), np.random.rand(grid_rows, grid_cols)), axis=2)

    gif_creator = AnimatedGIFCreator(gif_dir='temp_gif_frames', gif_file=gif_file)

    for iteration in range(num_iterations):
        learning_rate = (eta_start - eta_end) * (num_iterations - iteration) / (num_iterations - 1) + eta_end
        neighborhood_radius = int((radius_start - radius_end) * (num_iterations - iteration) / (num_iterations - 1) + radius_end)

        weights = update_weights(sample_data, weights, learning_rate, neighborhood_radius)

        plt.clf()
        plot_data(sample_data)
        plot_weights(weights)
        plt.title(f"Step: {iteration + 1}/{num_iterations}, Radius: {neighborhood_radius}, Learning Rate: {learning_rate:.2f}")
        plt.draw()
        gif_creator.add_plot_frame(plt.gcf())

    gif_creator.create_gif(duration=100, last_frame_duration=500)
    gif_creator.clear_frames()
plt.show()


GIF saved as C:\Users\ma\Desktop\新建文件夹\人工神经网络\第三次作业\矩形区域500点.gif
GIF saved as C:\Users\ma\Desktop\新建文件夹\人工神经网络\第三次作业\矩形区域200点.gif
GIF saved as C:\Users\ma\Desktop\新建文件夹\人工神经网络\第三次作业\矩形区域2000点.gif
GIF saved as C:\Users\ma\Desktop\新建文件夹\人工神经网络\第三次作业\矩形区域200点小区域.gif


# 解决旅行商问题

In [9]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import amin, array, where
from PIL import Image
import os
import random

class AnimatedGIFCreator:
    def __init__(self, gif_dir='temp_gif_frames', gif_file='output.gif'):
        self.gif_dir = gif_dir
        self.gif_file = gif_file
        self.frames = []
        self.frame_count = 0
        self._create_gif_directory()

    def _create_gif_directory(self):
        if not os.path.exists(self.gif_dir):
            os.makedirs(self.gif_dir)

    def add_plot_frame(self, plt_figure=None):
        plt_figure = plt_figure or plt.gcf()
        filename = os.path.join(self.gif_dir, f"frame_{self.frame_count:04d}.png")
        plt_figure.savefig(filename, bbox_inches='tight')
        self.frames.append(Image.open(filename))
        self.frame_count += 1
        plt.close(plt_figure)

    def create_gif(self, duration=100, last_frame_duration=200):
        if not self.frames:
            print("No frames to save.")
            return

        durations = [duration] * (len(self.frames) - 1) + [last_frame_duration]
        self.frames[0].save(
            self.gif_file,
            save_all=True,
            append_images=self.frames[1:],
            duration=durations,
            loop=0
        )
        print(f"GIF saved as {self.gif_file}")

    def clear_frames(self):
        for frame in os.listdir(self.gif_dir):
            os.remove(os.path.join(self.gif_dir, frame))
        self.frames = []
        self.frame_count = 0

# Normalizing input data points
city_coordinates = np.array([[236, 53], [408, 79], [909, 89], [115, 264], [396, 335], 
                             [185, 456], [699, 252], [963, 317], [922, 389], [649, 515]])
city_coordinates_normalized = city_coordinates / np.array([1000, 600])

def plot_data(data_points, show_connections=False, plot_title=''):
    plt.scatter(data_points[:, 0], data_points[:, 1], s=10 if not show_connections else 35, 
                c='blue' if not show_connections else 'red', 
                label='City Coordinates' if not show_connections else 'Neuron Positions')
    if show_connections:
        plt.plot(data_points[:, 0], data_points[:, 1], 'y--', linewidth=1)
    plt.xlabel("x1")
    plt.ylabel("x2")
    plt.axis([-0.05, 1.05, -0.05, 1.05])
    if plot_title:
        plt.title(plot_title)
    plt.grid(True)
    plt.legend(loc='upper right')
    plt.tight_layout()

def find_winner(input_vector, weight_matrix):
    distances = np.sum((weight_matrix - input_vector) ** 2, axis=1)
    return np.argmin(distances)

def find_neighbors_1d(winner_index, num_neurons, neighborhood_radius):
    if neighborhood_radius <= 0:
        return [winner_index]
    return [(winner_index + offset) % num_neurons for offset in range(-neighborhood_radius, neighborhood_radius + 1)]

def update_weights(input_data, weight_matrix, learning_rate, neighborhood_radius):
    for input_vector in input_data:
        winner_index = find_winner(input_vector, weight_matrix)
        neighborhood = find_neighbors_1d(winner_index, weight_matrix.shape[0], neighborhood_radius)
        for neighbor_index in neighborhood:
            noise = (np.random.rand(2) - 0.5) * learning_rate ** 2  # Add random noise for better convergence
            weight_matrix[neighbor_index] += learning_rate * (input_vector - weight_matrix[neighbor_index]) + noise
    return weight_matrix

# Configuration
num_samples = city_coordinates_normalized.shape[0]
num_neurons = num_samples
num_iterations = 200
learning_rate_start = 0.3
learning_rate_end = 0.01
radius_start = 2
radius_end = 0

# Initialize weights
neuron_positions = np.random.rand(num_neurons, city_coordinates_normalized.shape[1])

# Create GIF
gif_creator = AnimatedGIFCreator(gif_dir='temp_gif_frames', gif_file='C:\\Users\\ma\\Desktop\\新建文件夹\\人工神经网络\\第三次作业\\旅行商问题.gif')

# Training Loop
for iteration in range(num_iterations):
    learning_rate = learning_rate_start + (learning_rate_end - learning_rate_start) * iteration / (num_iterations - 1)
    neighborhood_radius = int(radius_start + (radius_end - radius_start) * iteration / (num_iterations - 1))
    
    neuron_positions = update_weights(city_coordinates_normalized, neuron_positions, learning_rate, neighborhood_radius)

    plt.clf()
    plot_data(city_coordinates_normalized, show_connections=False)
    plot_data(neuron_positions, show_connections=True, plot_title=f"Step: {iteration + 1}/{num_iterations}, Radius: {neighborhood_radius}, Learning Rate: {learning_rate:.2f}")
    plt.draw()
    gif_creator.add_plot_frame(plt.gcf())

# Save and clear GIF frames
gif_creator.create_gif(duration=100, last_frame_duration=500)
gif_creator.clear_frames()
plt.show()


GIF saved as C:\Users\ma\Desktop\新建文件夹\人工神经网络\第三次作业\旅行商问题.gif


In [10]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import amin, array, where
from PIL import Image
import os
import random

class AnimatedGIFCreator:
    def __init__(self, gif_dir='temp_gif_frames', gif_file='output.gif'):
        self.gif_dir = gif_dir
        self.gif_file = gif_file
        self.frames = []
        self.frame_count = 0
        self._create_gif_directory()

    def _create_gif_directory(self):
        if not os.path.exists(self.gif_dir):
            os.makedirs(self.gif_dir)

    def add_plot_frame(self, plt_figure=None):
        plt_figure = plt_figure or plt.gcf()
        filename = os.path.join(self.gif_dir, f"frame_{self.frame_count:04d}.png")
        plt_figure.savefig(filename, bbox_inches='tight')
        self.frames.append(Image.open(filename))
        self.frame_count += 1
        plt.close(plt_figure)

    def save_gif(self, duration=100, last_frame_duration=200):
        if not self.frames:
            print("No frames to save.")
            return

        durations = [duration] * (len(self.frames) - 1) + [last_frame_duration]
        self.frames[0].save(
            self.gif_file,
            save_all=True,
            append_images=self.frames[1:],
            duration=durations,
            loop=0
        )
        print(f"GIF saved as {self.gif_file}")

    def clear_frames(self):
        for frame in os.listdir(self.gif_dir):
            os.remove(os.path.join(self.gif_dir, frame))
        self.frames = []
        self.frame_count = 0

def normalize_data(data, max_values):
    return data / max_values

def plot_data(data_points, show_connections=False, plot_title=''):
    plt.scatter(data_points[:, 0], data_points[:, 1], s=10 if not show_connections else 35, 
                c='blue' if not show_connections else 'red', 
                label='View Site' if not show_connections else 'Neuron Positions')
    if show_connections:
        plt.plot(data_points[:, 0], data_points[:, 1], 'y--', linewidth=1)
    plt.xlabel("x1")
    plt.ylabel("x2")
    plt.axis([0, 1.5, 0, 1.5])
    if plot_title:
        plt.title(plot_title)
    plt.grid(True)
    plt.legend(loc='upper right')
    plt.tight_layout()

def find_winner_node(input_vector, weight_matrix):
    distances = np.sum((weight_matrix - input_vector) ** 2, axis=1)
    return np.argmin(distances)

def find_neighbors(winner_index, num_neurons, neighborhood_radius):
    if neighborhood_radius <= 0:
        return [winner_index]
    return [(winner_index + offset) % num_neurons for offset in range(-neighborhood_radius, neighborhood_radius + 1)]

def update_weights(input_data, weight_matrix, learning_rate, neighborhood_radius):
    for input_vector in input_data:
        winner_index = find_winner_node(input_vector, weight_matrix)
        neighborhood = find_neighbors(winner_index, weight_matrix.shape[0], neighborhood_radius)
        for neighbor_index in neighborhood:
            noise = (np.random.rand(2) - 0.5) * learning_rate ** 2  # Add random noise for better convergence
            weight_matrix[neighbor_index] += learning_rate * (input_vector - weight_matrix[neighbor_index]) + noise
    return weight_matrix

def train_and_generate_gif(input_data, gif_filename, num_iterations=200, learning_rate_start=0.3, learning_rate_end=0.01, radius_start=2, radius_end=0):
    num_samples = input_data.shape[0]
    weight_matrix = np.random.rand(num_samples, input_data.shape[1])

    gif_creator = AnimatedGIFCreator(gif_dir='temp_gif_frames', gif_file=gif_filename)

    for iteration in range(num_iterations):
        learning_rate = learning_rate_start + (learning_rate_end - learning_rate_start) * iteration / (num_iterations - 1)
        neighborhood_radius = int(radius_start + (radius_end - radius_start) * iteration / (num_iterations - 1))

        weight_matrix = update_weights(input_data, weight_matrix, learning_rate, neighborhood_radius)

        plt.clf()
        plot_data(input_data, show_connections=False)
        plot_data(weight_matrix, show_connections=True, plot_title=f"Step: {iteration + 1}/{num_iterations}, Radius: {neighborhood_radius}, Learning Rate: {learning_rate:.2f}")
        plt.draw()
        gif_creator.add_plot_frame(plt.gcf())

    gif_creator.save_gif(duration=100, last_frame_duration=500)
    gif_creator.clear_frames()
    plt.show()

# Input Data
city_coordinates_1 = np.array([
    [349, 198], [268, 510], [736, 381], [1048, 187], [924, 480],
    [969, 682], [1034, 793], [597, 754], [631, 556], [173, 304]
])
city_coordinates_2 = np.array([
    [297, 338], [403, 604], [736, 381], [1039, 286], [668, 553],
    [929, 598], [900, 137], [606, 761], [304, 448], [521, 430]
])
city_coordinates_3 = np.array([
    [369, 170], [713, 415], [742, 600], [828, 325], [876, 675],
    [106, 340], [1038, 803], [845, 823], [1165, 151], [546, 814]
])

# Normalize Data
city_coordinates_1 = normalize_data(city_coordinates_1, [1000, 600])
city_coordinates_2 = normalize_data(city_coordinates_2, [1000, 600])
city_coordinates_3 = normalize_data(city_coordinates_3, [1000, 600])

# Train and Generate GIFs
train_and_generate_gif(city_coordinates_1, 'C:\\Users\\ma\\Desktop\\新建文件夹\\人工神经网络\\第三次作业\\第一种分布.gif')
train_and_generate_gif(city_coordinates_2, 'C:\\Users\\ma\\Desktop\\新建文件夹\\人工神经网络\\第三次作业\\第二种分布.gif')
train_and_generate_gif(city_coordinates_3, 'C:\\Users\\ma\\Desktop\\新建文件夹\\人工神经网络\\第三次作业\\第三种分布.gif')


GIF saved as C:\Users\ma\Desktop\新建文件夹\人工神经网络\第三次作业\第一种分布.gif
GIF saved as C:\Users\ma\Desktop\新建文件夹\人工神经网络\第三次作业\第二种分布.gif
GIF saved as C:\Users\ma\Desktop\新建文件夹\人工神经网络\第三次作业\第三种分布.gif
